In [42]:
#you might need to install some of these libraries and/or modules before trying to use them
#I typed in and ran !pip install [insert library name] and then pip install [insert library name] again in another cell. I also had to restart my Kernel.
import numpy as np
import pandas as pd
import re
import nltk
import nltk.corpus
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
import collections

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/curlhomefolder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/curlhomefolder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/curlhomefolder/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [41]:
#you might need to install some of these libraries and/or modules before trying to use them
#I typed in and ran !pip install [insert library name] and then pip install [insert library name] again in another cell. I also had to restart my Kernel. 
#These 
from transformers import pipeline
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [7]:
#import a csv file containg comments from reddit and store in a data frame. 
BPD_medicine_read = pd.read_csv("Desktop/Erdos/Data-science-project-Mental-Health/Curls_Data_Scraping/rBPD_Data/medicine_comments.csv")
BPD_medicine_read

,Unnamed: 0,Comment,Author,Post
0,0,Medication isn't used for BPD. There is no app...,MainAction9667,1b92zml
1,1,Bupropion/Wellbutrin has been really good for ...,NaN,1b92zml
2,2,"I've tried most medicines, mostly SSRIs, mood ...",erbstar,1b92zml
3,3,"Unfortunately, there are no meds specifically ...",soundofdarkness1987,1b92zml
4,4,Therapy>medication,terrorshark503,1b92zml
...,...,...,...,...
2586,15,"Oh, I'm sorry, I thought you were a male, sinc...",kb3uoe,k2lzw8
2587,0,"Can relate, been on 7 different medications an...",NaN,wsc065
2588,0,Welcome! Sounds like you're already heading in...,flaaffi,mbi0fr
2589,1,"Hey nice to ""meet you"" Michael, im new here to...",emeraldgreeneye,mbi0fr


In [8]:
#Get rid of the rows that contain no comments
BPD_medicine_read.dropna(subset=['Comment'], inplace=True)
BPD_medicine_read
#there don't happen to be any rows without comments. However, what do we want to do with NaN authors? 

,Unnamed: 0,Comment,Author,Post
0,0,Medication isn't used for BPD. There is no app...,MainAction9667,1b92zml
1,1,Bupropion/Wellbutrin has been really good for ...,NaN,1b92zml
2,2,"I've tried most medicines, mostly SSRIs, mood ...",erbstar,1b92zml
3,3,"Unfortunately, there are no meds specifically ...",soundofdarkness1987,1b92zml
4,4,Therapy>medication,terrorshark503,1b92zml
...,...,...,...,...
2586,15,"Oh, I'm sorry, I thought you were a male, sinc...",kb3uoe,k2lzw8
2587,0,"Can relate, been on 7 different medications an...",NaN,wsc065
2588,0,Welcome! Sounds like you're already heading in...,flaaffi,mbi0fr
2589,1,"Hey nice to ""meet you"" Michael, im new here to...",emeraldgreeneye,mbi0fr


In [9]:
#define a function that should clean any string of text entered. 
def clean_text(text):
    #convert all cases of letters to lower case
    text = text.lower()
    #get rid of characters that are not English words 
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    #function will return cleaned text 
    return text

In [10]:
#clean the comments in the data frame and update the "Comment" column. 
BPD_medicine_read["Comment"] = BPD_medicine_read["Comment"].apply(clean_text)
BPD_medicine_read

,Unnamed: 0,Comment,Author,Post
0,0,medication isnt used for bpd there is no appro...,MainAction9667,1b92zml
1,1,bupropionwellbutrin has been really good for m...,NaN,1b92zml
2,2,ive tried most medicines mostly ssris mood sta...,erbstar,1b92zml
3,3,unfortunately there are no meds specifically f...,soundofdarkness1987,1b92zml
4,4,therapymedication,terrorshark503,1b92zml
...,...,...,...,...
2586,15,oh im sorry i thought you were a male since yo...,kb3uoe,k2lzw8
2587,0,can relate been on different medications and y...,NaN,wsc065
2588,0,welcome sounds like youre already heading in t...,flaaffi,mbi0fr
2589,1,hey nice to meet you michael im new here too i...,emeraldgreeneye,mbi0fr


In [12]:
#Append all the comments from the "Comment" column to a single list
list_of_comments = BPD_medicine_read["Comment"].tolist()

In [13]:
## Bag of words/Count Vectorization 
# CountVectorizwer is method to convert text to numerical data. You feed in a string and a list of the unique words used in the string is returned as well as the number of times each word appears.
# import CountVectorizer from sklearn library 
from sklearn.feature_extraction.text import CountVectorizer
#give a name to the list of strings we want to convert to data. Note that "corpus" means a collection of written texts, especially the entire works of a particular author or a body of writing on a particular subject.
corpus = list_of_comments
#give a name to CountVectorizer and we tell it to ignore English language "stop words." A stop word is a commonly used word (such as “the”, “a”, “an”, or “in”) that a search engine has been programmed to ignore. So,
#CountVectorizer will ignore all stop words. 
countvectorizer = CountVectorizer(stop_words='english')
#apply CountVectorizer to each of the strings in the list
X = countvectorizer.fit_transform(corpus)
#print out the names of the unque words from all the strings
print(countvectorizer.get_feature_names_out())
#convert the transformed text into a matrix or array.
result = X.toarray()
#convert the matrix or array to a data frame with the unique words as the column headings. 
df_bow_sklearn = pd.DataFrame(result,columns=countvectorizer.get_feature_names_out())
df_bow_sklearn

['aana' 'aba' 'abandon' ... 'zonked' 'zookeeper' 'zoplicon']


,aana,aba,abandon,abandoned,abandoning,abandonment,abilify,abilifys,abilities,ability,...,zinc,zizek,zolfresh,zoloft,zombie,zombierobot,zone,zonked,zookeeper,zoplicon
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
##TF-IDF Vectorization
#TF-IDF is a weighting system that assigns a weight to each word in a document based on its term frequency (tf) and the reciprocal document frequency (tf) (idf)
#import TfidfVectorizer from sklearn library 
from sklearn.feature_extraction.text import TfidfVectorizer
#give a name to TfidfVectorizer and we tell it to ignore English language stop words. 
vectorizer = TfidfVectorizer(stop_words='english')
#apply TfidfVectorizer to each of the strings in the list
vectors = vectorizer.fit_transform(corpus)
#give a name to the unque words from all the strings
feature_names = vectorizer.get_feature_names_out()
#print out all of the unique words 
print(f"Feature names \n{feature_names}")
#Store a dense representation of the NDFrame [NDFrame is an N-dimensional analogue of DataFrame] which means that memory will be allocated to store even the missing values in the dataframe. 
matrix = vectors.todense()
#Convert the NDFrame to a list of lists 
list_dense = matrix.tolist()
#Convert list of lists to a data frame with the unique words as the column headings.
df = pd.DataFrame(list_dense,columns=feature_names)
df

Feature names 
['aana' 'aba' 'abandon' ... 'zonked' 'zookeeper' 'zoplicon']


,aana,aba,abandon,abandoned,abandoning,abandonment,abilify,abilifys,abilities,ability,...,zinc,zizek,zolfresh,zoloft,zombie,zombierobot,zone,zonked,zookeeper,zoplicon
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
#Another vectorization technique that I thought of myself but is pretty similar to the above
#create a list of key phrases to check for in a string
keyphrases = ["i'm", "i take", "i've been taking", "i go to", "go to" , "been taking", "i've been on", "been on", "i have", "i was diagnosed", "prescribed", "i was prescribed", "i suffer from",
"i am", "i'm", "has been really good for me", "has been very good for me", "has been good for me", "has worked" , "has improved" ,"works for me", "i have bpd", "i have borderline personality disorder",
"major depressive disorder", "depression", "bipolar disorder","dysthymia","eating disorder", "antisocial personality disorder" , "aspd", "narcissistic personality disorder", "bpd has been"]
#Import the csv file that contains the list of keywords and turn it into a list 
keyword_file = pd.read_csv("Desktop/Erdos/Data-science-project-Mental-Health/Curls_Data_Scraping/keywords.csv")
keywords = keyword_file['therapy'].tolist()
#create a list of keys by combining the the keyphrases and keywords into one list 
keys = keyphrases + keywords

In [45]:
#define a function with arguments being a string, "comment," and another string, "word" 
def check_comment_create_score(comment,word):
#check to see if the the "word" string is in the "comment" string
#if "word" is in "comment" create a variable called phrase_score and assign 1 as its value
    if word in comment:
        phrase_score = 1
#if "word" is not in "comment" create a variable called phrase_score and assign 0 as its value
    else:
        phrase_score = 0
#the function will return the phrase score for the specified word and comment 
    return phrase_score

#define a function with arguments being a list of strings "keyword_list" and another list of strings "comment_list"
def create_phrase_matrix(keyword_list,comment_list):
#create a list of lists by a doubly iterated list comprehension for each comment we apply the previous function and check to see if each word in the keyword list is in the comment or not. 
    p_matrix = [[check_comment_create_score(comment,word) for word in keyword_list] for comment in comment_list]
#create a data frame of the previous list of lists with column headers being the strings in the keyword list
    df_of_matrix = pd.DataFrame(p_matrix, columns = keyword_list) 
#the function returns the data frame 
    return df_of_matrix

In [18]:
#returns a data frame where each comment in "list_of_comments" is checked for each word in "keys". The data frame contains a vector of phrase_scores for each comment with the entries of the vectors corresponding to the words in the key list.
create_phrase_matrix(keys,list_of_comments)

,i'm,i take,i've been taking,i go to,go to,been taking,i've been on,been on,i have,i was diagnosed,...,atarax,vistaril,antihistamine,anxiety medication,wellbutrin,buproprion,ndri,treatment,treat,side effects
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2587,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
#using this sentiment analysis from Hugging Face it seems I can't return beyond 90 sentiment scores for 90 comments
#Then, an object of the pipeline function is created and the task to be performed is passed as an argument (i.e sentiment analysis in our case). We can also specify the model that we need to use to perform the task.
#since we have not mentioned the model to be used, the distilbert-base-uncased-finetuned-sst-2-english mode is used by default for sentiment analysis
sentiment_pipeline = pipeline("sentiment-analysis")
#create a list comprehension of labels and sentiment scores for each comment in the list. I'm only able to do 90 comments for some reason. 
list_of_comments_part1 = [sentiment_pipeline(list_of_comments[i]) for i in range(0,91)]
list_of_comments_part1

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/envs/erdos_may_2024_v2/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


[[{'label': 'NEGATIVE', 'score': 0.9969779253005981}],
 [{'label': 'NEGATIVE', 'score': 0.9978169202804565}],
 [{'label': 'NEGATIVE', 'score': 0.9821292757987976}],
 [{'label': 'NEGATIVE', 'score': 0.963953971862793}],
 [{'label': 'NEGATIVE', 'score': 0.6644101142883301}],
 [{'label': 'NEGATIVE', 'score': 0.669893205165863}],
 [{'label': 'NEGATIVE', 'score': 0.9918580651283264}],
 [{'label': 'NEGATIVE', 'score': 0.9989914298057556}],
 [{'label': 'NEGATIVE', 'score': 0.9993452429771423}],
 [{'label': 'POSITIVE', 'score': 0.9965983033180237}],
 [{'label': 'NEGATIVE', 'score': 0.9137454628944397}],
 [{'label': 'POSITIVE', 'score': 0.9898157119750977}],
 [{'label': 'NEGATIVE', 'score': 0.9995542168617249}],
 [{'label': 'POSITIVE', 'score': 0.9982395172119141}],
 [{'label': 'NEGATIVE', 'score': 0.9969872832298279}],
 [{'label': 'NEGATIVE', 'score': 0.9979668259620667}],
 [{'label': 'NEGATIVE', 'score': 0.9955174326896667}],
 [{'label': 'NEGATIVE', 'score': 0.9992710947990417}],
 [{'label': 

In [31]:
#this is another sentiment analysis from Hugging Face it seems I can't return beyond 90 sentiment scores for 90 comments for this one either
distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)
list_of_comments_part2 = [distilled_student_sentiment_classifier(list_of_comments[i]) for i in range(0,91)]
list_of_comments_part2

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


[[[{'label': 'positive', 'score': 0.23920847475528717},
   {'label': 'neutral', 'score': 0.32259440422058105},
   {'label': 'negative', 'score': 0.43819716572761536}]],
 [[{'label': 'positive', 'score': 0.29631951451301575},
   {'label': 'neutral', 'score': 0.15029390156269073},
   {'label': 'negative', 'score': 0.5533865690231323}]],
 [[{'label': 'positive', 'score': 0.4894973337650299},
   {'label': 'neutral', 'score': 0.17352990806102753},
   {'label': 'negative', 'score': 0.3369726836681366}]],
 [[{'label': 'positive', 'score': 0.4368133544921875},
   {'label': 'neutral', 'score': 0.21805936098098755},
   {'label': 'negative', 'score': 0.34512728452682495}]],
 [[{'label': 'positive', 'score': 0.6115221977233887},
   {'label': 'neutral', 'score': 0.22695645689964294},
   {'label': 'negative', 'score': 0.1615213304758072}]],
 [[{'label': 'positive', 'score': 0.21680070459842682},
   {'label': 'neutral', 'score': 0.165004700422287},
   {'label': 'negative', 'score': 0.6181946396827698

In [48]:
#TextBlob sentiment analysis
#Polarity determines the sentiment of the text. Its values lie in [-1,1] where -1 denotes a highly negative sentiment and 1 denotes a highly positive sentiment.
#Subjectivity determines whether a text input is factual information or a personal opinion. Its value lies between [0,1] where a value closer to 0 denotes a piece of factual information and a value closer to 1 denotes a personal opinion.
#another list comprehension is used here to generate the polarity of the comments. I am able to generate a value for all comments. 
list_of_comments_blob_polarity = [TextBlob(list_of_comments[i]).sentiment.polarity for i in range(0,len(list_of_comments))]
list_of_comments_blob_polarity

[0.16666666666666666,
 0.1132716049382716,
 0.47500000000000003,
 0.23333333333333336,
 0.0,
 0.17876984126984125,
 0.3333333333333333,
 0.125,
 0.0,
 0.0,
 -0.0008503401360544346,
 0.4,
 0.4,
 0.16777777777777775,
 0.27666666666666667,
 -0.0066666666666666706,
 0.3333333333333333,
 0.2857142857142857,
 0.19999999999999998,
 0.025000000000000005,
 0.07278338945005614,
 -0.09444444444444444,
 0.0,
 0.2333333333333333,
 1.0,
 0.21818181818181817,
 -0.4333333333333333,
 -0.39375000000000004,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.014285714285714282,
 -0.3,
 0.0,
 0.26666666666666666,
 0.8,
 0.0,
 0.0,
 0.25,
 0.0,
 0.0,
 0.15306878306878305,
 0.17500000000000002,
 -0.1875,
 0.0,
 -0.18333333333333335,
 0.06137566137566137,
 0.1,
 0.25606060606060604,
 0.5,
 0.328125,
 -0.16070075757575758,
 0.005555555555555561,
 0.13333333333333333,
 0.0,
 0.8,
 0.0,
 -0.13777777777777778,
 0.0,
 0.050476190476190494,
 0.35000000000000003,
 -0.02777777777777779,
 0.11041666666666666,
 0.36,
 0.7,
 0.0,
 0.1444

In [49]:
#another list comprehension is used here to generate the subjectivity of the comments. I am able to generate a value for all comments.
list_of_comments_blob_subjectivity = [TextBlob(list_of_comments[i]).sentiment.subjectivity for i in range(0,len(list_of_comments))]
list_of_comments_blob_subjectivity

[0.7000000000000001,
 0.5956790123456791,
 0.44166666666666665,
 0.5243589743589744,
 0.0,
 0.6343253968253968,
 0.5,
 0.5,
 0.0,
 0.05,
 0.6824829931972788,
 0.59,
 0.4,
 0.2547222222222222,
 0.69,
 0.3952380952380952,
 0.4333333333333333,
 0.5357142857142857,
 0.45555555555555555,
 0.5666666666666668,
 0.5519640852974186,
 0.36666666666666664,
 0.0,
 0.3444444444444444,
 0.3,
 0.2772727272727273,
 0.9,
 0.54375,
 0.06666666666666667,
 0.3,
 0.3,
 0.5,
 0.0,
 0.6714285714285715,
 0.4,
 0.0,
 0.4444444444444444,
 0.8,
 0.0,
 0.0,
 0.4666666666666667,
 0.0,
 0.0,
 0.6492063492063492,
 0.4,
 0.7083333333333334,
 0.0,
 0.6888888888888888,
 0.6224867724867725,
 0.4,
 0.29444444444444445,
 0.5,
 0.59375,
 0.6010416666666666,
 0.7277777777777779,
 0.42500000000000004,
 0.0,
 0.75,
 0.0,
 0.5872222222222221,
 0.0,
 0.40714285714285714,
 0.44999999999999996,
 0.3333333333333334,
 0.5645833333333334,
 0.52,
 0.6000000000000001,
 0.0,
 0.15555555555555556,
 0.48,
 0.5816666666666668,
 0.0,
 0.66

In [51]:
#VADER (Valence Aware Dictionary and Sentiment Reasoner) is a rule-based sentiment analyzer that has been trained on social media text
#VaderSentiment object returns a dictionary of sentiment scores for the text to be analyzed
vadersentiment = SentimentIntensityAnalyzer()
#another list comprehension is used to generate the dictionary for each comment and store it in a list
list_of_comments_vader_polarity = [vadersentiment.polarity_scores(list_of_comments[i]) for i in range(0,len(list_of_comments))]
list_of_comments_vader_polarity

[{'neg': 0.015, 'neu': 0.967, 'pos': 0.019, 'compound': 0.1114},
 {'neg': 0.146, 'neu': 0.742, 'pos': 0.113, 'compound': -0.9458},
 {'neg': 0.0, 'neu': 0.939, 'pos': 0.061, 'compound': 0.3818},
 {'neg': 0.036, 'neu': 0.892, 'pos': 0.071, 'compound': 0.8539},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.032, 'neu': 0.862, 'pos': 0.106, 'compound': 0.8934},
 {'neg': 0.0, 'neu': 0.842, 'pos': 0.158, 'compound': 0.7876},
 {'neg': 0.093, 'neu': 0.622, 'pos': 0.285, 'compound': 0.5719},
 {'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'compound': 0.3818},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.043, 'neu': 0.867, 'pos': 0.09, 'compound': 0.5994},
 {'neg': 0.0, 'neu': 0.586, 'pos': 0.414, 'compound': 0.9142},
 {'neg': 0.303, 'neu': 0.697, 'pos': 0.0, 'compound': -0.6715},
 {'neg': 0.017, 'neu': 0.868, 'pos': 0.115, 'compound': 0.8187},
 {'neg': 0.123, 'neu': 0.702, 'pos': 0.175, 'compound': 0.5238},
 {'neg': 0.033, 'neu': 0.929, 'pos': 0.038, 'compoun